In [ ]:
import sys

#Due to leftoevererrors in Nathan's python installation, some cleaning up occurs here
#sys.path.append("./code/")
#sys.path.remove('/usr/local/lib/python2.7/site-packages') 


import matplotlib.pyplot as plt
%matplotlib inline
import copy
import datetime
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats



from scipy.stats import norm
from sklearn.neighbors import KernelDensity


import itertools
#sys.path


import regex as re
import datetime
import math
import time
import itertools

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff

import plotly
plotly.tools.set_credentials_file(username='juliettapc', api_key='nM6iUdx6dGaOiPXQTwpP')   # go to: https://plot.ly/settings/api#/   for a new key if needed

########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [ ]:
                                  

####  NEW FILE INCLUDING EARLY CITATIONS OF YOUNG REFERENCES:   ../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit.pkl
                                            
#%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_added_more_columns_no_self-cit_one_ref_per_sect_ONLY_ARTICLES.pkl', 'rb'))



%time df_merged = pickle.load(open('../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit_and_after_accretion_time.pkl', 'rb'))
print ("done loading pickles", df_merged.shape)




df_merged = df_merged[df_merged['cite_count'] != -1]   # i dont know why, but there are 7 occurrences with value -1



plos_df = df_merged.drop_duplicates(subset=['paper_UT'])
print (plos_df.shape)



df_merged.head()

In [ ]:


####  FIGURE 4B:  histograms  NUMBER OF CIATIONS OF A PAPER VS # CITATIONS OF ITS YOUNGGGGG REFERENCES
#### and NEW distplot (smoothing with kernel density estimation) with collapsed histograms of the same data



string_top_bottom_plos  = 'bottom'  # top or bottom (plos papers)
      

years=[2011]      #[2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017] 

  
    
   
    

string_isolated_ref = ""   #""   #"  #""   # 0  or 1 (or empty string, to include all ref)
string_self_ref =0         #""      # ""   #1   # 0  or 1 (or empty string, to include all ref)   OJO!!! THIS NEW FILE DOES NOT INCLUDE SELF-CITATIONS TO BEGING WITH
string_code_categ="" #  ojo!!! the codes are strings, not integers. if i want to include multiple subjects:  "1 2 8"
string_journal=""#   PLOS ONE"
string_plos_field=""#['D CU BIOLOGY']"




preselection_df = df_merged.drop_duplicates(subset=['paper_UT', 'reference_UT'])
print ("size of preselection (each ref. only used once per plos paper):",preselection_df.shape)




    

##### preselection by plos year
print (years)
preselection_df = preselection_df[preselection_df['plos_pub_year'].isin(years)]  
print ("size of preselection1 (by plos years):",preselection_df.shape)





#### i remove self-citations
preselection_df = preselection_df[preselection_df['self_citation']== string_self_ref ]  
print ("size of preselection (no self-cit):",preselection_df.shape)
   



######### preselection by isolated or group references:
if (string_isolated_ref==0) or  ( string_isolated_ref == 1 ): 
    preselection_df0 = preselection_df[preselection_df['isolated_citation']== string_isolated_ref ]  
else:    
    preselection_df0 = preselection_df   
    print ("size of preselection1 (by isolated/group ref):",preselection_df0.shape, string_isolated_ref)








######### preselection by plos ONE subject category:
if string_code_categ=="": 
    preselection_df111 = preselection_df0
else:    
    if " " not in string_code_categ:  # to include one single category
        preselection_df111 = preselection_df0[preselection_df0['categ_codes'].str.contains(string_code_categ)]        
     
    else:  # if multiple codes-categories
        list_codes = string_code_categ.split(" ")
        

        if len(list_codes) >= 2:              
            preselection_df111 = preselection_df0[ preselection_df0['categ_codes'].str.contains('|'.join(list_codes)) ]  # to look for partial matches from a list of strings!!!!       


    print (" size of preselection (by plos ONE subject category):",preselection_df111.shape, string_code_categ)








######### preselection by plos journal:
if string_journal=="": 
    preselection_df1 = preselection_df111
else:    
    preselection_df1 = preselection_df111[preselection_df111['plos_j1']== string_journal ]  
print (" size of preselection2 (by plos journal):",preselection_df1.shape, string_journal)







######### preselection by plos field:
if string_plos_field=="": 
    preselection_df2 = preselection_df1
else:    
    preselection_df2 = preselection_df1[preselection_df1['plos_field']== string_plos_field ]  
print (" size of preselection2 (by plos field):",preselection_df2.shape, string_plos_field)





#### i select only young ref!
time_window = 1
preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] >= (min(years)-time_window) ]   
print (" size of preselection3 (only young ref):",preselection_df3.shape, "   # unique plos papers:", len(preselection_df3.paper_UT.unique()))
       




# print ("\nTot # records included:",len(preselection_df3),"   # plos papers:",len(preselection_df3.paper_UT.unique()),\
#        "   # unique ref:", len(preselection_df3.reference_UT.unique()),'\n')




  

#### ojo!!! bins for plos on unique plos records
df_plos = preselection_df3.drop_duplicates(subset=['paper_UT'])
#list_q=[0.3,0.6,.9,.99,1]
list_q=[0.1,0.6,.9,1]

quantiles=sorted(list(df_plos['paper_cite_count'].quantile(list_q).to_dict().items())) #mean 10.68 
 
# print (quantiles)   

lista_bins_plos_citations=[]
old_value=0
for item in quantiles:
    pair=[old_value, int(item[1])]
    lista_bins_plos_citations.append(pair)
    old_value = int(item[1])
    
# print (lista_bins_plos_citations)


   

    
   
list_subsets_data = []




fig = tools.make_subplots(rows=1, cols=4, vertical_spacing=0.001, horizontal_spacing=0.001)
cont_columns =1

list_median =[]
list_mean =[]
list_x_pos = []
list_annot_x = []
lista_x_series = []

if string_top_bottom_plos  == 'top' :
    
    color_hist =' #4d88ff'
    left_space = 200
    
    text_title = 'Top 10% papers'
    list_x_pos = [132,165,130,173]
    
    text_y_axis=""#Number citations <br>young references"
    
    #### first, i select only the top-plos papers
    item = lista_bins_plos_citations[-1]
    
    tot_minimo = item[0]
    tot_maximo = item[1]

    selection_df_top = preselection_df3[(preselection_df3['paper_cite_count'] >= tot_minimo)  &  (preselection_df3['paper_cite_count'] < tot_maximo)]

  
  
    print ("min, max in the top bin:",selection_df_top.paper_cite_count.min(), selection_df_top.paper_cite_count.max())
    
    
    ### i get 4 bins for the top-plos selection
    df_top_plos_only = selection_df_top.drop_duplicates(subset=['paper_UT'])
    
    list_q=[0.25,.5,.75,1]

    quantiles_top=sorted(list(df_top_plos_only['paper_cite_count'].quantile(list_q).to_dict().items())) #mean 10.68    
    list_bins_histogr=[]
    old_value=selection_df_top.paper_cite_count.min()
    for item in quantiles_top:
        pair=[old_value, int(item[1])]
        list_bins_histogr.append(pair)
        old_value = int(item[1])    
    
    print (string_top_bottom_plos, tot_minimo, tot_maximo, selection_df_top.shape, " # plos",   len(selection_df_top.paper_UT.unique()),\
             " #ref:",len(selection_df_top.reference_UT.unique()), "bins within top-plos:", list_bins_histogr)
     
        
    

    ## one histogram per bin of top-plos
    for item in list_bins_histogr :

        minimo = item[0]
        maximo = item[1]
        
        etiqueta =str(item[0])+"-"+str(item[1]-1)+" Citations"  #str(item).replace("]",'').replace("[",'').replace(", ",'-')+" Citations"
        list_annot_x.append(etiqueta)# = ['00 Citations', '01 Citation','02 Citations','03 Citations']

        df_selection = selection_df_top[(selection_df_top['paper_cite_count'] >= minimo)  &  (selection_df_top['paper_cite_count'] < maximo)]

        y = df_selection['cite_count']
        
    
        lista_x_series.append(y.tolist())
        
        
        
        trace1 = go.Histogram(
                    y=y,
                    name= str(minimo)+"-"+str(maximo)+ " citations, N:"+str(len(y)),                    
                    ybins=dict(
                           start=0,
                           end=selection_df_top.cite_count.max(),
                           size=10),
                     marker=dict(
                         color=color_hist ),                   
            showlegend = False, 
                   )

        fig.append_trace(trace1, 1, cont_columns)

        cont_columns +=1

        print ("   bin:",minimo, maximo, df_selection.shape, df_selection['cite_count'].mean(), df_selection['cite_count'].std())

        list_median.append(df_selection.cite_count.median())
        list_mean.append(df_selection.cite_count.mean())

        list_subsets_data.append(list(df_selection.cite_count.values))


elif string_top_bottom_plos  == 'bottom' :
    
    left_space = 350
    
    text_title = 'Bottom 10% papers'
    list_annot_x = ['0 Citations', '1 Citation','2 Citations','3 Citations']
    list_x_pos = [40,80,130,140]
    
    text_y_axis="Number of citations <br>of young references"
    
    color_hist = '#ff4d4d'
    
    ### first i select the bottom plos papers
    item = lista_bins_plos_citations[0]
    
    minimo = 0
    maximo = 4

    selection_df_bottom = preselection_df3[(preselection_df3['paper_cite_count'] >= minimo)  &  (preselection_df3['paper_cite_count'] < maximo)]

    print (string_top_bottom_plos, minimo, maximo, "  unique values in select", selection_df_bottom['paper_cite_count'].unique(), selection_df_bottom.shape, \
           " # plos:",len(selection_df_bottom.paper_UT.unique()), " # ref",len(selection_df_bottom.reference_UT.unique()))


    
    
    
    ### then i get one histogram per value
    for  value in range(4):#number of ciations of papers 0, 1,2,3, 4: #####################

                

        df_selection = selection_df_bottom[selection_df_bottom['paper_cite_count'] == value]   
        y = df_selection['cite_count']

        lista_x_series.append(y.tolist())
        
        trace1 = go.Histogram(
                    y=y,
                   # name= str(value)+ " citations, N:"+str(len(y)),
                    ybins=dict(
                           start=0,
                           end=selection_df_bottom.cite_count.max(),
                           size=10),
                    marker=dict(
                        color=color_hist,),                   
            showlegend = False, 
           
                )

        fig.append_trace(trace1, 1, cont_columns)
        cont_columns +=1

        print ("   bin:",value, df_selection.shape, df_selection['cite_count'].mean(), df_selection['cite_count'].std())

        
        list_median.append(df_selection.cite_count.median())
        list_mean.append(df_selection.cite_count.mean())
        list_subsets_data.append(list(df_selection.cite_count.values))
        
#fig.layout.yaxis.update({'title':'Number of citations of young references'}) 
#fig.layout.xaxis3.update({'title':'Number of citations of papers') 



fig.layout.yaxis1.update({'range':[-5,450]})  
fig.layout.yaxis2.update({'range':[-5,450]})  
fig.layout.yaxis3.update({'range':[-5,450]})  
fig.layout.yaxis4.update({'range':[-5,450]})  
        
    
     
    
fig.layout.yaxis2.update({'showticklabels':False})     
fig.layout.yaxis3.update({'showticklabels':False})     
fig.layout.yaxis4.update({'showticklabels':False})        


fig.layout.yaxis1.update({'gridwidth':5,'showgrid':True})       
fig.layout.yaxis2.update({'gridwidth':5,'showgrid':True})     
fig.layout.yaxis3.update({'gridwidth':5,'showgrid':True})     
fig.layout.yaxis4.update({'gridwidth':5,'showgrid':True})  


fig.layout.xaxis1.update({'showgrid':False})       
fig.layout.xaxis2.update({'showgrid':False})     
fig.layout.xaxis3.update({'showgrid':False})     
fig.layout.xaxis4.update({'showgrid':False})   


# layout = go.Layout(
#     xaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),



fig.layout.update(
    {
   
     'shapes': [
        # Line Horizontal --  median
        {
            'type': 'line',
            'xref': 'x',
            'yref': 'y',
            'x0': 0,
            'y0': list_median[0],
            'x1': list_x_pos[0]+7,
            'y1': list_median[0],
            'line': {
                'color': 'rgb(0,0,0)',
                'width': 8.5,
                
            },
        },
      # Line Horizontal --  median
        {
            'type': 'line',
            'xref': 'x2',
            'yref': 'y',
            'x0': 0,
            'y0': list_median[1],
            'x1': list_x_pos[1]+7,
            'y1': list_median[1],
            'line': {
                'color': 'rgb(0,0,0)',
                'width': 8.5,
               
            },
        },
        # Line Horizontal --  median
        {
            'type': 'line',
            'xref': 'x3',
            'yref': 'y',
            'x0': 0,
            'y0': list_median[2],
            'x1': list_x_pos[2]+15,
            'y1': list_median[2],
            'line': {
                'color': 'rgb(0,0,0)',
                'width': 8.5,
                
            },
        },
      # Line Horizontal --  median
        {
            'type': 'line',
            'xref': 'x4',
            'yref': 'y',
            'x0': 0,
            'y0': list_median[3],
            'x1': list_x_pos[3]+15,
            'y1': list_median[3],
            'line': {
                'color': 'rgb(0,0,0)',
                'width': 8.5,
                
            },
        },  
         
         
         
         
         
          # Line Horizontal --  mean
        {
            'type': 'line',
            'xref': 'x',
            'yref': 'y',
            'x0': 0,
            'y0': list_mean[0],
            'x1': list_x_pos[0]+7,
            'y1': list_mean[0],
            'line': {
                'color': 'rgb(0,0,0)',
                'width': 8.5,
                'dash': 'dot',
            },
        },
      # Line Horizontal --  mean
        {
            'type': 'line',
            'xref': 'x2',
            'yref': 'y',
            'x0': 0,
            'y0': list_mean[1],
            'x1': list_x_pos[1]+7,
            'y1': list_mean[1],
            'line': {
                'color': 'rgb(0,0,0)',
                'width': 8.5,
                'dash': 'dot',
            },
        },
        # Line Horizontal --  mean
        {
            'type': 'line',
            'xref': 'x3',
            'yref': 'y',
            'x0': 0,
            'y0': list_mean[2],
            'x1': list_x_pos[2]+15,
            'y1': list_mean[2],
            'line': {
                'color': 'rgb(0,0,0)',
                'width': 8.5,
                'dash': 'dot',
            },
        },
      # Line Horizontal --  mean
        {
            'type': 'line',
            'xref': 'x4',
            'yref': 'y',
            'x0': 0,
            'y0': list_mean[3],
            'x1': list_x_pos[3]+15,
            'y1': list_mean[3],
            'line': {
                'color': 'rgb(0,0,0)',
                'width': 8.5,
                'dash': 'dot',
            },
        },  
     
     
     
     ]  ,
        
     'annotations' : [   
         dict(  # title
            x=.5,
            y=1.3,  #1.15,
            xref='paper',
            yref='paper',
            text=text_title,
            showarrow=False,  
            font=dict(               
                size=65,),
           ),    
         
         
         dict(  # this is for the xaxis label
            x=.5,
            y=-.25,
            xref='paper',
            yref='paper',
            text="Number of citations",
            showarrow=False,  
            font=dict(               
                size=65,),
           ),    
         
         dict(  # this is for the yaxis label
            x=-.2,
            y=.5,
            xref='paper',
            yref='paper',
            text=text_y_axis,
            showarrow=False, 
            textangle=-90,
            font=dict(               
                size=65,),
           ),    
         
         
         
         
         
         
          dict(
            x=list_x_pos[0]/2.,
            y=1.1, #-.15,
            xref='x',
            yref='paper',
            text=list_annot_x[0],
            showarrow=False,  
              font=dict(               
                size=40,),
           ),    
         
          dict(
            x=list_x_pos[1]/2.,
            y=1.1, #-.15,
            xref='x2',
            yref='paper',
            text=list_annot_x[1],
            showarrow=False,
              font=dict(               
                size=40,),
           ),    
         
          dict(
            x=list_x_pos[2]/2.,
            y=1.1, #-.15,
            xref='x3',
            yref='paper',
            text=list_annot_x[2],
            showarrow=False,
              font=dict(               
                size=40,),
           ),    
         
          dict(
            x=list_x_pos[3]/2.,
            y=1.1, #-.15,
            xref='x4',
            yref='paper',
            text=list_annot_x[3],
            showarrow=False,
              font=dict(               
                size=40,),
           ),    
         
         
         
         
       ]
                 
                 
                 
    } )


print ("median:",list_median)

font_gral=35   # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral

# fig['layout']['yaxis']['titlefont']['size'] = font_gral 
      


fig['layout']['margin']=dict(
        l=left_space,
       # r=50,
        b=200,
        t=250,        
    )



offline.plot(fig, auto_open=True, image = 'png', image_filename='testing_multiple_vertical_histogr' ,image_width=1800, image_height=1200, filename='../plots/testing_multiple_vertical_histogr.html', validate=True)

###############################################
###############################################
###############################################
###############################################
###############################################








list_colors = ['#67a9cf','#91cf60','#f1a340','#d7191c']

#list_colors = ['#a6611a','#dfc27d','#80cdc1','#018571']
# if string_top_bottom_plos  == 'top' :
#     list_colors = ['#bdd7e7','#6baed6','#3182bd','#08519c']
# elif string_top_bottom_plos  == 'bottom' :
#     list_colors = ['#fdcc8a','#fc8d59','#e34a33','#b30000']


############  Create distplot with custom bin_size
fig = ff.create_distplot(lista_x_series, list_annot_x, bin_size=10,  show_hist=False,  show_rug=False, show_curve=True, colors =  list_colors)#,curve_type='lognormal')#, curve_type='kde', show_rug=False, bin_size=.2, , show_hist=False)




##################  Layout


fig['layout']['xaxis']['range'] = [1,500]  #[.001,3.5]  ## because the axis is log, the range needs to be in log units too! 
  
#fig['layout']['xaxis']['tickvals'] =  [1, 10, 100, 1000,2000]   
 
fig['layout']['xaxis']['title'] = "Number of citations"
fig['layout']['yaxis']['title'] = "Probability density"
fig['layout']['title'] = string_top_bottom_plos.capitalize()+" 10% papers"    
      

    
    
font_gral=50   # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral
fig['layout']['xaxis']['titlefont']['size'] = font_gral 
fig['layout']['yaxis']['titlefont']['size'] = font_gral 

fig['layout']['legend']=dict( x=0.75, y=.9,  ) #font=dict(size=20) 
                           




fig['layout']['margin']=dict(
        l=250,
        r=50,
        b=150,
        t=150,  
    )


    


    
# Plot!
offline.plot(fig, auto_open=True, image = 'png', image_filename='multiple_distplot' ,image_width=1800, image_height=1200, filename='../plots/multiple_distplot.html', validate=True)

### NOTE:  this figures is requires some touch up from the plotly platform (lines width cant be customized from here for some reason)

##################################################3
###################################################3

#### run old Figure 4b cell first to get the list of values!!!!


list_colors = ['#67a9cf','#91cf60','#f1a340','#d7191c']

data = []
for i in  range(len(lista_x_series)):
    
    a =  lista_x_series[i]
    c = list_colors[i]
    
    ### equivalent to normalized cumulative distribution  !!!
    x = np.sort(a)
    y = np.linspace(0, 1, len(a), endpoint=False)
    
    
    trace0 = go.Scatter(
        x = x,
        y = y,
        mode = 'lines',
        name = list_annot_x[i],
        marker = dict(         
                color = c,
                line = dict(
                        width = 50,
                        color = c
                )
            )
    )

    data.append(trace0)
    
        
    
    
    
    
    
layout = Layout(    


    xaxis=dict(
        title= "Number of citations received",   
#         titlefont=dict(
#             #size=font_axes,
#             color='black'),  
#         tickfont=dict(   
#             #family=font,
#             size=font_ticks,
#             color='black'),
         #range = [1, 500],#range=[np.log10(0.1),np.log10(max_x)],   # because the axis is log, the range needs to be in log units too!  :(
         type='log',            
        ),
    
    yaxis=dict(
        title='Probability density',
        #type='log',
#         titlefont=dict(            
#             size=font_axes,
#             color='black'
#         #    color='lightgrey'
#         ),  
#         tickfont=dict(   
#             #family=font,
#             size=font_ticks,
#             color='black'
#         ),
        ),                



)       


fig = dict(data=data, layout=layout)
    
fig['layout']['xaxis']['title'] = "Number of citations"
fig['layout']['yaxis']['title'] = "Cumulative probability density"
fig['layout']['title'] = string_top_bottom_plos.capitalize()+" 10% papers"    
      

    
    
font_gral=50   # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral
fig['layout']['xaxis']['titlefont']['size'] = font_gral +10
fig['layout']['yaxis']['titlefont']['size'] = font_gral +10

fig['layout']['legend']=dict( x=0.75, y=.9,  ) #font=dict(size=20) 
                           




fig['layout']['margin']=dict(
        l=250,
        r=50,
        b=150,
        t=150,  
    )    
    
    
    
    
    
    
    
    
    
    
    
    
    

# Plot!
offline.plot(fig, auto_open=True, image = 'png', image_filename='histogram_num_cit_ref_'+string_top_bottom_plos+str(10)+'_'+str(years[0]) ,image_width=1800, image_height=1200, filename='../plots/histogram.html', validate=True)


In [ ]:
#######################  MORE SOPHISTICATED RANDOMIZATION SCHEME: ALSO CONTROLING FOR PLOS FIELD  (AS WELL AS PLOS YEAR)  AND PRESERVING REFERENCES THAT ARE CITED TOGETHER IN A GROUP
############################################## ####################### ####################### ####################### ####################### ####################### 


### Figure 4A



#### (i compare the usage of top/nontop references by top/nontop plos papers with a null model that comes from randomizing the data)








#### (i compare the usafe of top/nontop references by top/nontop plos papers with a null model that comes from randomizing the data)

Niter=1000

list_list_years = [[2008],[2009],[2010],[2011],[2012],[2013],[2014]]

#years=[2008]
    
for years in list_list_years:
   

    string_references_age="all"   #young"#old"  # young # all   for the selection of what references i include
    string_isolated_ref=""  #""   # 0  or 1 (or empty string, to include all ref)
    string_self_ref=0    #""#1   # 0  or 1 (or empty string, to include all ref)







    ######### plos ONE categories. 
    string_code_categ="0" #  ojo!!! the codes are strings, not integers. if i want to include multiple subjects:  "1 2 8"

    #  '0': 'Biology and life sciences'             6,032,537
    #  '1': 'Computer and information sciences'     1,207,799
    #  '10': 'Social sciences'                      755,899
    #  '2': 'Earth sciences'                        533,155
    #  '3': 'Ecology and environmental sciences'    624,142
    #  '4': 'Engineering and technology'            382,247 
    #  '5': 'Medicine and health sciences'          4,535,926   
    #  '6': 'People and places'                     691,523
    #  '7': 'Physical sciences'                     2,100,827
    #  '8': 'Research and analysis methods'         3,871,470
    #  '9': 'Science policy'                        43,360 






    # for string_code_categ in range(11):
    #     string_code_categ = str(string_code_categ)









    ######### plos journals 
    string_journal=""

        # PLOS ONE       6,367,070
        # PLOS GENET      149,923
        # PLO NE TR D     138,289   # (neglected tropical diseases)
        # PLOS PATHOG     109,803
        # PLOS COMPUT      77,924
        # PLOS BIOL        56,754
        # PLOS MED         24,506



    ######### WoS subject categories. 
    string_plos_field=""#['D CU BIOLOGY']"

    # ['D RO MULTIDISCIPLINARY SCIENCES']                                                                                                       4464540
    # ['D CU BIOLOGY']                                                                                                                          1055045
    # ['D RO MULTIDISCIPLINARY SCIENCES', 'D CU BIOLOGY']                                                                                        847485
    # ['D KM GENETICS & HEREDITY']                                                                                                               149923
    # ['D YU TROPICAL MEDICINE', 'D TI PARASITOLOGY']                                                                                            138289
    # ['D ZE VIROLOGY', 'D QU MICROBIOLOGY', 'D TI PARASITOLOGY']                                                                                109803
    # ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY']                                                          77687
    # ['D CQ BIOCHEMISTRY & MOLECULAR BIOLOGY', 'D CU BIOLOGY']                                                                                   56754
    # ['D PY MEDICINE, GENERAL & INTERNAL']                                                                                                       24506
    # ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY', 'D PO MATHEMATICS, INTERDISCIPLINARY APPLICATIONS']        237











    print ("original size:",df_merged.shape)








    ##### preselection by plos year
    print (years)
    preselection_df = df_merged[df_merged['plos_pub_year'].isin(years)]  
    print ("size of preselection1 (by plos years):",preselection_df.shape)





    #### i remove self-citations
    if (string_self_ref==0) or  ( string_self_ref == 1 ): 
        preselection_df = preselection_df[preselection_df['self_citation']== string_self_ref ]  
        if string_self_ref ==0:
            string_self_ref = ", no self-cit"
        elif string_self_ref ==1:
            string_self_ref = ", only self-cit"





    ######### preselection by isolated or group references:
    if (string_isolated_ref==0) or  ( string_isolated_ref == 1 ): 
        preselection_df0 = preselection_df[preselection_df['isolated_citation']== string_isolated_ref ]  

        if string_isolated_ref ==0:
            string_isolated_ref = ", group ref"
        elif string_isolated_ref ==1:
            string_isolated_ref = ", isolated ref"
    else:    
        preselection_df0 = preselection_df   
        print ("size of preselection1 (by isolated/group ref):",preselection_df0.shape, string_isolated_ref)








    ######### preselection by plos ONE subject category:
    if string_code_categ=="": 
        preselection_df111 = preselection_df0
    else:    
        if " " not in string_code_categ:  # to include one single category
            preselection_df111 = preselection_df0[preselection_df0['categ_codes'].str.contains(string_code_categ)]        
            string_code_categ = " "+dict_code_categ[string_code_categ]  

        else:  # if multiple codes-categories
            list_codes = string_code_categ.split(" ")
            print (list_codes)

            if len(list_codes) >= 2:              
                preselection_df111 = preselection_df0[ preselection_df0['categ_codes'].str.contains('|'.join(list_codes)) ]  # to look for partial matches from a list of strings!!!!!


            string_code_categ = "" 
            for code in list_codes:
                string_code_categ += "-"+dict_code_categ[code] 


        print (" size of preselection (by plos ONE subject category):",preselection_df111.shape, string_code_categ)








    ######### preselection by plos journal:
    if string_journal=="": 
        preselection_df1 = preselection_df111
    else:    
        preselection_df1 = preselection_df111[preselection_df111['plos_j1']== string_journal ]  
    print (" size of preselection2 (by plos journal):",preselection_df1.shape, string_journal)







    ######### preselection by plos field:
    if string_plos_field=="": 
        preselection_df2 = preselection_df1
    else:    
        preselection_df2 = preselection_df1[preselection_df1['plos_field']== string_plos_field ]  
    print (" size of preselection2 (by plos field):",preselection_df2.shape, string_plos_field)





    ##### preselection only young/old references:       
    preselection_df3 = preselection_df2
    if string_references_age == "young":
        time_window_age = 1   
        preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] >= (min(years)-time_window_age) ]   

        print ("    size of preselection3 (only young references):",preselection_df3.shape, string_references_age)

    elif string_references_age == "old":
        time_window_age = 10    
        preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] <= (min(years)-time_window_age) ]   

        print ("    size of preselection3 (only old references):",preselection_df3.shape,string_references_age )





    N_plos = len(preselection_df3.paper_UT.unique())         
    N_ref = len(preselection_df3.reference_UT.unique()) 
    N_all = len(preselection_df3)




    print ("     N plos:", N_plos,"  N  ref:",N_ref, " N records:", N_all)        



    preselection_df3 = preselection_df3.drop_duplicates(subset=['paper_UT', 'reference_UT'])

    print ("OJO!!! EACH REFERENCE ONLY COUNTED ONCE PER PAPER:")

    N_plos = len(preselection_df3.paper_UT.unique())         
    N_ref = len(preselection_df3.reference_UT.unique()) 
    N_all = len(preselection_df3)




    print ("     N plos:", N_plos,"  N  ref:",N_ref, " N records:", N_all)        














    ############## i define quantiles for plos papers (for that subselection, and based on their FINAL number of citations):
    list_q_plos=[.1,.9,1]
    #list_q_plos=[.05,.95,1]

    df_for_quantiles_plos = preselection_df3.drop_duplicates(subset=['paper_UT'])   # ojo!!! dont use preselection_df3 directly because there are REPETITIONS!!!!

    quantiles=sorted(list(df_for_quantiles_plos['paper_cite_count'].quantile(list_q_plos).to_dict().items())) #mean 10.68 

    print ("\n\ncitation bins for the selected plos:", list_q_plos)#,quantiles, df_for_quantiles_plos.shape)   

    lista_bins_plos=[]
    old_value=0
    for item in quantiles:
        pair=[old_value, int(item[1])]
        lista_bins_plos.append(pair)
        old_value = int(item[1])

    #print (lista_bins_plos, min(preselection_df3['paper_cite_count']), max(preselection_df3['paper_cite_count']))

    print ("\nbins for PLOS papers:")

    cont = 0
    dict_bin_list_plos_UT={}
    for item in lista_bins_plos:

        minimo = item[0]
        maximo = item[1]   

        df_select = preselection_df3[(preselection_df3['paper_cite_count'] >= minimo)  &  (preselection_df3['paper_cite_count'] < maximo)]
        llave=str(minimo)+"-"+str(maximo)
        dict_bin_list_plos_UT[llave]= list(df_select.paper_UT.unique())
        print (" ",llave, "  N:",len(list(df_select.reference_UT.unique())), "  avg # ref:",df_select.drop_duplicates(subset=['paper_UT']).total_refs.mean())
        max_key_plos=llave


        if cont ==0:
            min_key_plos = llave
            string_range_bottom_plos =   "["+llave+"] citations"


        cont  +=1



        string_range_top_plos =  "["+llave+"] citations"  

    print ("string for bottom:", string_range_bottom_plos, " string for top:", string_range_top_plos)




    ########## i define quantiles for references (based on their FINAL number of citations)
    #list_q_ref=[.2,.8,1]
    list_q_ref=[.1,.9,1]

    #list_q_ref=[.05,.95,1]
    df_for_quantiles_ref = preselection_df3.drop_duplicates(subset=['reference_UT'])   # ojo!!! remember to remove REPETITIONS!!!!
    quantiles=sorted(list(df_for_quantiles_ref['cite_count'].quantile(list_q_ref).to_dict().items())) #mean 10.68 

    print ("\n\ncitation bins for the references in the selected plos:", list_q_ref,quantiles)    

    lista_bins=[]
    old_value=0
    for item in quantiles:
        pair=[old_value, int(item[1])]
        lista_bins.append(pair)
        old_value = int(item[1])




    print ("\nbins for refrences:")


    cont = 0
    dict_bin_list_ref_UT={}
    for item in lista_bins:

        minimo = item[0]
        maximo = item[1]    

        df_select = preselection_df3[(preselection_df3['cite_count'] >= minimo)  &  (preselection_df3['cite_count'] < maximo)]
        llave=str(minimo)+"-"+str(maximo)
        dict_bin_list_ref_UT[llave]=list(df_select.reference_UT.unique())
        print (" ",llave, "N:",len(list(df_select.reference_UT.unique())), "  avg # ref:",df_select.drop_duplicates(subset=['reference_UT']).total_refs.mean())
        max_key_ref=llave

        if cont ==0:
            min_key_ref = llave
        cont  +=1




    ############### i create the list of top plos, top ref, bottom plos and bottom ref:
    #########################

    lista_top_plos = dict_bin_list_plos_UT[max_key_plos]
    print ("\n\n# UTs top",(100-100*list_q_plos[-2]),"% plos:",len(lista_top_plos))

    lista_top_ref=dict_bin_list_ref_UT[max_key_ref]
    print ("# UTs top",(100-100*list_q_ref[-2]),"% ref:", len(lista_top_ref))


    lista_bottom_plos = dict_bin_list_plos_UT[min_key_plos]
    print ("# UTs bottom ",(100*list_q_plos[0]),"% plos:",len(lista_bottom_plos))

    lista_bottom_ref=dict_bin_list_ref_UT[min_key_ref]
    print ("# UTs bottom ",(100*list_q_ref[0]),"% ref:", len(lista_bottom_ref))

    list_plos_in_year= list(preselection_df3.paper_UT.unique())
    print ("Tot # records:",len(preselection_df3),", # plos:",len(list_plos_in_year))





    ######  i look at the usage of the top ref
    ################################################  

    df_top_ref = preselection_df3[preselection_df3['reference_UT'].isin(lista_top_ref)]


    df_top_ref_top_plos = df_top_ref[df_top_ref['paper_UT'].isin(lista_top_plos)]
    df_top_ref_bottom_plos = df_top_ref[df_top_ref['paper_UT'].isin(lista_bottom_plos)]


    usage_top_ref_top_plos = len(df_top_ref_top_plos)/float(len(df_top_ref))
    usage_top_ref_bottom_plos = len(df_top_ref_bottom_plos)/float(len(df_top_ref))


    print ("fraction of usage of top ref by ")
    print ("  top",(100-100*list_q_plos[-2]),"% plos:",  usage_top_ref_top_plos)
    print ("  bottom",(100*list_q_plos[0]),"% plos:",usage_top_ref_bottom_plos  )







    ######  i look at the usage of the non-top ref
    ################################################      

    df_non_top_ref = preselection_df3[preselection_df3['reference_UT'].isin(lista_bottom_ref)]


    df_non_top_ref_top_plos = df_non_top_ref[df_non_top_ref['paper_UT'].isin(lista_top_plos)]
    df_non_top_ref_bottom_plos = df_non_top_ref[df_non_top_ref['paper_UT'].isin(lista_bottom_plos)]

    usage_non_top_ref_top_plos = len(df_non_top_ref_top_plos)/float(len(df_non_top_ref))
    usage_non_top_ref_bottom_plos = len(df_non_top_ref_bottom_plos)/float(len(df_non_top_ref))


    print ("fraction of usage of non-top ref by ")
    print ("  top",(100-100*list_q_plos[-2]),"% plos:", usage_non_top_ref_top_plos )
    print ("  bottom",(100*list_q_plos[0]),"% plos:", usage_non_top_ref_bottom_plos )






    ####################
    # I canculate the null model (usage of references by top and non top plos papers, from the randomized data)
    #################################################  

    lista_usage_top_ref_by_top_plos_rand = []
    lista_usage_top_ref_by_bottom_plos_rand = []

    lista_usage_nontop_ref_by_top_plos_rand = []
    lista_usage_nontop_ref_by_bottom_plos_rand = []





    #### first i get the list of lists corresponding to the references used in the selected df, preserving the reference grouping or isolation:  
    lista_lists_values = get_list_lists_references(preselection_df3)

    print ("len list_lists_all_ref:",len(lista_lists_values), preselection_df3.shape)


    for i in range(Niter):

        print (i)



        #### old, simple randomization scheme (only controling for year)
    #     lista_values = list(preselection_df3.reference_UT)   #[i for i in range (len(df_merged))]
    #     random.shuffle(lista_values)
    #     preselection_df3['randomized_ref_UT'] = lista_values






        ########   new randomization scheme (controling for year, but also preserving groups of references cited together in a paper):   
        #### lista_values is created outside the Niter loop   (i only need to do it once per selected df)    



        random.shuffle(lista_lists_values)      
        ### to flat out a list of lists:   
        flat_list = []    
        for sublist in lista_lists_values:
            for item in sublist:
                flat_list.append(item)


       # print ("len flat list:", len(flat_list), preselection_df3.shape)

        preselection_df3['randomized_ref_UT'] = flat_list   ### ojo!!! esto randomiza paper_UT, no reference_UT  !!!!!!








        ####### (RANDOMIZED)  i look at the usage of the top ref
        df_top_ref_rand = preselection_df3[preselection_df3['randomized_ref_UT'].isin(lista_top_ref)]

        df_top_ref_top_plos_rand = df_top_ref_rand[df_top_ref_rand['paper_UT'].isin(lista_top_plos)]
        df_top_ref_bottom_plos_rand = df_top_ref_rand[df_top_ref_rand['paper_UT'].isin(lista_bottom_plos)]


        usage_top_ref_top_plos_rand = len(df_top_ref_top_plos_rand)/float(len(df_top_ref_rand))
        usage_top_ref_bottom_plos_rand = len(df_top_ref_bottom_plos_rand)/float(len(df_top_ref_rand))


        lista_usage_top_ref_by_top_plos_rand.append(usage_top_ref_top_plos_rand)
        lista_usage_top_ref_by_bottom_plos_rand.append(usage_top_ref_bottom_plos_rand)






        #######  (RANDOMIZED) i look at the usage of the non-top ref            
        df_non_top_ref_rand = preselection_df3[preselection_df3['randomized_ref_UT'].isin(lista_bottom_ref)]        

        df_non_top_ref_top_plos_rand = df_non_top_ref_rand[df_non_top_ref_rand['paper_UT'].isin(lista_top_plos)]
        df_non_top_ref_bottom_plos_rand = df_non_top_ref_rand[df_non_top_ref_rand['paper_UT'].isin(lista_bottom_plos)]


        usage_non_top_ref_top_plos_rand = len(df_non_top_ref_top_plos_rand)/float(len(df_non_top_ref_rand))
        usage_non_top_ref_bottom_plos_rand = len(df_non_top_ref_bottom_plos_rand)/float(len(df_non_top_ref_rand))


        lista_usage_nontop_ref_by_top_plos_rand.append(usage_non_top_ref_top_plos_rand)
        lista_usage_nontop_ref_by_bottom_plos_rand.append(usage_non_top_ref_bottom_plos_rand)




    print("plos category:", string_code_categ)




    print ("\n\n\n\navg randomized!!")
    print("fraction of usage of top ref by")
    print ("  top",(100-100*list_q_plos[-2]),"% plos:",  np.mean(lista_usage_top_ref_by_top_plos_rand) )   
    print ("  bottom",(100*list_q_plos[0]),"% plos:",np.mean(lista_usage_top_ref_by_bottom_plos_rand)  )




    print ("\n\navg randomized")
    print ("fraction of usage of non-top ref by ")
    print ("  top",(100-100*list_q_plos[-2]),"% plos:", np.mean(lista_usage_nontop_ref_by_top_plos_rand) )   
    print ("  bottom",(100*list_q_plos[0]),"% plos:", np.mean(lista_usage_nontop_ref_by_bottom_plos_rand) ,"\n\n\n")












    #############################  second part


    ###  PLOT FIGURE FOR MORE SOPHISTICATED VERSION OF THE RANDOMIZATION SCHEME
    ######     (RUN PREVIOUS CELL FIRST, TO GET THE BOOT-STRAPPING DATA)


    ### group by Top PApers  Bottom Papers




    #########################################

    #lista_bin_names=['Top '+str(int(100-100*list_q_plos[-2]))+'%<br>papers', 'Bottom '+str(int(list_q_plos[0]*100))+'%<br>papers']
    #lista_bin_names = ['Bottom '+str(int(list_q_plos[0]*100))+'%<br>papers', 'Top '+str(int(100-100*list_q_plos[-2]))+'%<br>papers']

    lista_bin_names = ['Bottom '+str(int(list_q_plos[0]*100))+'% papers<br>'+string_range_bottom_plos, 'Top '+str(int(100-100*list_q_plos[-2]))+'% papers<br>'+string_range_top_plos]






    lista_for_top_ref = [ usage_top_ref_top_plos, usage_top_ref_bottom_plos]
    lista_for_bottom_ref = [usage_non_top_ref_top_plos, usage_non_top_ref_bottom_plos]




    lista_for_top_ref = [  usage_top_ref_bottom_plos, usage_top_ref_top_plos]
    lista_for_bottom_ref = [usage_non_top_ref_bottom_plos, usage_non_top_ref_top_plos]





    ###### this is the null model 


    # lista_expectations_top_ref = [np.mean(lista_usage_top_ref_by_top_plos_rand),np.mean(lista_usage_top_ref_by_bottom_plos_rand)]  
    # lista_expectations_bottom_ref = [np.mean(lista_usage_nontop_ref_by_top_plos_rand), np.mean(lista_usage_nontop_ref_by_bottom_plos_rand)] 


    lista_expectations_top_ref = [np.mean(lista_usage_top_ref_by_bottom_plos_rand), np.mean(lista_usage_top_ref_by_top_plos_rand)]  
    lista_expectations_bottom_ref = [ np.mean(lista_usage_nontop_ref_by_bottom_plos_rand),np.mean(lista_usage_nontop_ref_by_top_plos_rand)] 






    # list_errors_top_ref = [2.*np.std(lista_usage_top_ref_by_top_plos_rand), 2.*np.std(lista_usage_top_ref_by_bottom_plos_rand)] 
    # list_errors_bottom_ref = [2.*np.std(lista_usage_nontop_ref_by_top_plos_rand) , 2.*np.std(lista_usage_nontop_ref_by_bottom_plos_rand)] 


    list_errors_top_ref = [2.*np.std(lista_usage_top_ref_by_bottom_plos_rand), 2.*np.std(lista_usage_top_ref_by_top_plos_rand) ] 
    list_errors_bottom_ref = [2.*np.std(lista_usage_nontop_ref_by_bottom_plos_rand), 2.*np.std(lista_usage_nontop_ref_by_top_plos_rand) ] 






    z_score_top_ref_by_top_plos = (usage_top_ref_top_plos - np.mean(lista_usage_top_ref_by_top_plos_rand))/np.std(lista_usage_top_ref_by_top_plos_rand)
    z_score_nontop_ref_by_top_plos = (usage_non_top_ref_top_plos - np.mean(lista_usage_nontop_ref_by_top_plos_rand))/np.std(lista_usage_nontop_ref_by_top_plos_rand)

    z_score_top_ref_by_bottom_plos = (usage_top_ref_bottom_plos - np.mean(lista_usage_top_ref_by_bottom_plos_rand))/np.std(lista_usage_top_ref_by_bottom_plos_rand)
    z_score_nontop_ref_by_bottom_plos = (usage_non_top_ref_bottom_plos - np.mean(lista_usage_nontop_ref_by_bottom_plos_rand))/np.std(lista_usage_nontop_ref_by_bottom_plos_rand)




    print ('zscore top ref cited  by top plos:', z_score_top_ref_by_top_plos)
    print ('zscore bottom ref cited  by top plos:', z_score_nontop_ref_by_top_plos)

    print ('zscore top ref cited  by bottom plos:', z_score_top_ref_by_bottom_plos)
    print ('zscore bottom ref cited  by bottom plos:', z_score_nontop_ref_by_bottom_plos)



    title_string=''#'s top-ref by top plos: '+str(z_score_top_ref_by_top_plos)+';  zs top-ref by bottom plos: '+str(z_score_top_ref_by_bottom_plos)+\
    #'<br>zs nontop-ref by top plos: '+str(z_score_nontop_ref_by_top_plos)+';  zs nontop-ref by bottom plos: '+str(z_score_nontop_ref_by_bottom_plos)



    size_bar_name = 30#45
    y_pos_bar_names = -.039
    angle = -70

    trace1 = go.Bar(
        x=lista_bin_names,
        y=lista_for_top_ref,
    #     text=['by top '+str(int(100-100*list_q_plos[-2]))+'% papers', 'by top '+str(int(100-100*list_q_plos[-2]))+'% papers'],    
    #     name='by top '+str(int(100-100*list_q_plos[-2]))+'% papers',   
        marker=dict(
            color='#88419d',           
        ),



    )



    trace2 = go.Bar(
        x=lista_bin_names,
        y=lista_expectations_top_ref,
    #     name='   expected value',
    #     text=['Expected value for top', 'Expected value for top'],  
        error_y=dict(
           # type='data',
            array=list_errors_top_ref,
            thickness=5,
            visible=True
        ),
        marker=dict(
            color='#c994c7',         
        ),
    )


    trace3 = go.Bar(
        x=lista_bin_names,
        y=lista_for_bottom_ref,
    #     name='by bottom '+str(int(list_q_plos[0]*100))+'% papers', 
    #     text=['by bottom '+str(int(list_q_plos[0]*100))+'% papers','by bottom '+str(int(list_q_plos[0]*100))+'% papers'],
        marker=dict(
            color='#225ea8',   
        ),
    )


    trace4 = go.Bar(
        x=lista_bin_names,
        y=lista_expectations_bottom_ref,
      #  name='   expected value',

        error_y=dict(       
            array=list_errors_bottom_ref,#[0.5, 1, 2],
            thickness=5,
            visible=True
        ),
        marker=dict(
            color='#a6bddb',     
        ),
    )





    data = [trace1, trace2, trace3, trace4]
    layout = go.Layout(   
        title=title_string,
        xaxis = dict(
            side= 'top',
            range = [-.5,1.5],
           # showline =  True,
            #title= 'Plos Citation percentile'),
        ),
        yaxis = dict(
            title= 'Fraction references cited',
            range = [-.07,0.17],
            tickvals=[0.0,0.05,0.1,0.15],
            #showline =  True,
             ),

        showlegend=False,
    #     legend=dict(x=0.75, y=1.05,                 
    #                font=dict(
    #                     #family='sans-serif',
    #                     size=40,
    #                     #color='#000'
    #                     ),
    #                 ),


    #     barmode='stacked',#group',
       # bargap=0.2,
        bargroupgap=0.15,

        annotations = [  
            # the four bars on the left
            dict(
              x = -.34,
              y = y_pos_bar_names,
              text = 'Top '+str(int(100-100*list_q_ref[-2]))+'%<br>references',
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),    
            dict(
              x = -0.14,
              y = y_pos_bar_names,
              text = 'Null Model',
               textangle=angle,
                font = dict(size = size_bar_name ),
               ),
            dict(
              x = .08,
              y = y_pos_bar_names,
              showarrow = False,
              text = 'Bottom '+str(int(list_q_ref[0]*100))+'%<br>references', 
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),
            dict(
              x = .28,
              y = y_pos_bar_names,
              text =  'Null Model',
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),



            # the four bars on the right
            dict(  
              x = .68,
              y = y_pos_bar_names,
              text = 'Top '+str(int(100-100*list_q_ref[-2]))+'%<br>references',
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),
            dict(
              x = .88,
              y = y_pos_bar_names,
              text = 'Null Model',
               textangle=angle,
                font = dict(size = size_bar_name ),
               ),
            dict(
              x = 1.08,
              y = y_pos_bar_names,
              text = 'Bottom '+str(int(list_q_ref[0]*100))+'%<br>references', 
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),
            dict(
              x = 1.28,
              y = y_pos_bar_names,
              text =  'Null Model',
              textangle=angle,
                font = dict(size = size_bar_name ),
               ),


            ],    
    )








    fig = go.Figure(data=data, layout=layout)




    font_gral=55  # 20 if i wanna see it on the browser, 40 if i care about the png output
    fig['layout']['font']['size'] = font_gral-5


    # Altering x axis
    #fig['layout']['xaxis']['tickfont']['family'] = 'Gill Sans MT'
    fig['layout']['xaxis']['tickangle'] = 0
    # fig['layout']['yaxis']['tickangle'] = -90
    # fig['layout']['xaxis']['titlefont']['size'] = font_gral -10
    # fig['layout']['yaxis']['titlefont']['size'] = font_gral -10

    fig['layout']['xaxis']['tickfont']['size'] = font_gral -5
    fig['layout']['yaxis']['tickfont']['size'] = font_gral -10




    fig['layout']['margin']=dict(
            l=300,
           # r=50,
            b=100,
            t=200,
            pad=15
        )




    py.iplot(fig, filename='grouped-bar')


    fig_filename='fract_usage_top_bottom_ref_'+str(years[0])+string_code_categ.replace(" ","_")
    offline.plot(fig, auto_open=True, image = 'png', image_filename='../plots/'+fig_filename ,image_width=2000, image_height=1200, filename='../plots/'+fig_filename+'.html', validate=True)





    # earth science, 2013, 1000 iter, simple randomization scheme:

    # zscore top ref cited  by top plos: 1.8633822051
    # zscore bottom ref cited  by top plos: -8.29803601221
    # zscore top ref cited  by bottom plos: -4.18320566755
    # zscore bottom ref cited  by bottom plos: 2.22458205576





    ## earth science, 2013, 1000 iter, more sophisticated randomization scheme (preserving reference groups/isolated references):

    # zscore top ref cited  by top plos: 1.61069883113
    # zscore bottom ref cited  by top plos: -8.02168733553
    # zscore top ref cited  by bottom plos: -3.84738913187
    # zscore bottom ref cited  by bottom plos: 2.16556097748
